In [208]:
# The goal is to make it easy to import data
# for the self-driving car.
#
# To do this - this notebook collects raw data
# which has been collected into files, and package
# it into into a single python structure.
# It then saves this structure to
# a file that that can be easily imported.
#

In [209]:
import os 
subdir_base = os.path.dirname(os.path.realpath("data_out"))
subdir_list = [
    'pictures_01_two_loops_left',
    'pictures_02_three_loops_right',
    'pictures_03_three_loops_left',
    'pictures_04_four_loops_right',
#    'pictures_test',
]

print(subdir_base)
print(subdir_list)

/home/chris/Documents/projects_scikit_tensorflow/metrowest_scikit_tensorflow_cnn_car/data_out
['pictures_01_two_loops_left', 'pictures_02_three_loops_right', 'pictures_03_three_loops_left', 'pictures_04_four_loops_right']


In [210]:
# get a list of file numbers within a directory
# context - it is expected that files come in pairs named as:
#    controlNNN.txt
#    imageNNN.jpg
# The routine returns a list of NNNs

import fnmatch
def get_file_numbers(basedir, subdir):
    fullpath = os.path.join(basedir, subdir)
    # get all the file names
    names = os.listdir(fullpath)
    # filter to get a list of just the "control" files
    bar = fnmatch.filter(names, "control*")
    # use regular expression to get just the NNN part of the name
    m = [re.search('\d+', b).group(0) for b in bar]
    # convert to integer from string
    m = [ int(x) for x in m ]
    return m



In [211]:
# read raw images and control from files
# concatenate into a dataset

import os
import re
import numpy as np
from PIL import Image

my_dataset = {}
my_dataset['DESCR'] = """

    The dataset is 800 image and steering values from a toy
    car as it is driven around a track.  The track is a
    piece of white tape on a concrete floor. The steering values
    are the steering applied by the driver to keep the
    car on the track.
    
    The dictionary structure of the dataset is:
    
    "images" - A list of JPEG images.
    Each is a w=640 h=480 numpy.ndarray with RGB encoding.
    
    "steering" - A list of strings which indicate the direction
    the car is being steered at the time of the image is taken.
    These have a value ['left', 'right', 'straight']

    "target_names" - a list of values of the steering class ['left', 'right', 'straight']
    
    "DESCR" - an overview description of the dataset.
"""

my_dataset['target_names'] = ['left', 'right']
my_dataset['images'] = []
my_dataset['target'] = []

# loop through the directories/files
my_images = []
my_control = []
for subdir in subdir_list:
    filenumbers = get_file_numbers(subdir_base, subdir)
    for fnum in filenumbers:
        #print(f)
        
        filepath = os.path.join(subdir_base, subdir)

        filename = os.path.join(filepath, 'image' + str(fnum) + '.jpg')
        with Image.open(filename) as file:
            a = np.asarray(file)
            my_images.append(a)
            
#            print(file.format)            
#            [w,h] = file.size
#            print("width " + str(w) + " height " + str(h))
#            my_images.append(file)
#            file.show()

        filename = os.path.join(filepath, 'control' + str(fnum))
        with open(filename, 'rt') as file:
            line = file.read()
            direction = "straight"
            if line == "01":
                direction = "left"
            elif line == "10":
                direction = "right"
            my_control.append(direction)

my_dataset['images'] = my_images
my_dataset['target'] = my_control
my_dataset['DESCR'] = 'This is a description...'
my_dataset['target_names'] = ['left', 'right']

print('There are ' + str(len(my_dataset['images'])) + ' samples')
print(type(my_dataset))
#print(my_dataset.keys())
#print(type(my_dataset['images']))
print(type(my_dataset['images'][0]))
print(my_dataset['images'][0].size)  # (width, height) tuple
print(my_dataset['target'][0])


There are 806 samples
<class 'dict'>
<class 'numpy.ndarray'>
921600
straight


In [ ]:
# save to file
import pickle

with open("my_dataset.pickle", 'wb') as output:
    pickle.dump(my_dataset, output)

In [191]:
# load from file
import pickle

with open('my_dataset.pickle', 'rb') as data:
    dataset_in = pickle.load(data)

print('There are ' + str(len(dataset_in['images'])) + ' samples')
#print(type(dataset_in))
print(dataset_in.keys())
#print(type(dataset_in['images']))
print(type(dataset_in['images'][0]))
print(dataset_in['images'][0].size)  # (width, height) tuple   
print(my_dataset['target'][0])

There are 806 samples
dict_keys(['target_names', 'images', 'DESCR', 'target'])
<class 'numpy.ndarray'>
921600
straight


In [192]:
# with image and steering arrays as given (globals)
# the function takes in an index to those arrays
# convert nd_arry to PIL Image, and display
# it then uses PIL to draw lines forming an arrow
# to the left/right/forward based on steering
from PIL import Image, ImageDraw
import os
import time

def display_example(img_num):
    i_ndarray = my_dataset['images'][img_num]
    #print(type(i_ndarray))
    im = Image.fromarray(i_ndarray)
    #size = 128, 128
    #im.thumbnail(size)

    draw = ImageDraw.Draw(im)

    # establish 4 points and pointers... format for line sequence is is [(x,y),(x,y)...]
    hig   =  (im.size[0]/2,    im.size[1]/4)
    left  =  (im.size[0]/4,    im.size[1]/2)
    right =  (im.size[0]*3/4,  im.size[1]/2)
    low   =  (im.size[0]/2,    im.size[1]*3/4)
    pointer_left     = [low, left, high]
    pointer_straight = [left, high, right]
    pointer_right    = [high, right, low]
    pointer_box      = [high, right, low, left, high]

    steering = my_dataset['target'][img_num] 
    if steering == "left":
        pointer = pointer_left
    elif steering == "right":
        pointer = pointer_right
    elif steering == "straight":
        pointer = pointer_straight
    else:
        pointer = pointer_box
 
    draw.line(pointer)
    im.show()
    

In [200]:
import random
#import sleep
import time


print (len(my_dataset['images']))

for i in range(10):
    x = random.randrange(len(my_dataset['images']))
    print(x)
    display_example(x)
    time.sleep(3)

806
258
610
377
352
402
341
276
49
279
151
